In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import scipy.sparse as sps
import matplotlib.pyplot as plt

In [2]:
URM_file = open('data/train.csv', 'r')

def rowSplit (rowString):
    split = rowString.split(',')
    split[0] = int(split[0])
    split[1] = int(split[1])
    result = tuple(split)
    return result

next(URM_file)

URM_tuples = []
for line in URM_file:
    URM_tuples.append(rowSplit(line))
    
URM_tuples[0:10]

playlist_list, track_list = zip(*URM_tuples)

playlist_list = list(playlist_list)
track_list = list(track_list)

playlist_unique = list(set(playlist_list))
track_unique = list(set(track_list))

ratings_list = np.ones(len(playlist_list))

URM_all = sps.coo_matrix((ratings_list, (playlist_list, track_list)))
URM_all = URM_all.tocsr()

from Notebooks_utils.data_splitter import train_test_holdout

URM_train, URM_test = train_test_holdout(URM_all, train_perc = 0.8)

In [3]:
URM_train

<50446x20635 sparse matrix of type '<class 'numpy.float64'>'
	with 969203 stored elements in Compressed Sparse Row format>

In [4]:
URM_test

<50446x20635 sparse matrix of type '<class 'numpy.float64'>'
	with 242588 stored elements in Compressed Sparse Row format>

<h3>Step 1: Sampling</h3>

In [5]:
n_playlists = URM_train.shape[0]
n_tracks = URM_train.shape[1]

eligiblePlaylists = []

for playlist_id in range(n_playlists):
    
    start_pos = URM_train.indptr[playlist_id]
    end_pos = URM_train.indptr[playlist_id+1]
    
    if len(URM_train.data[start_pos:end_pos]) > 0:
        eligiblePlaylists.append(playlist_id)
        
print("Eligible playlists: {} out of {}".format(len(eligiblePlaylists), n_playlists))

Eligible playlists: 50445 out of 50446


In [6]:
def sampleTriplet():
    playlist_id = np.random.choice(n_playlists)
    
    playlist_seen_items = URM_train[playlist_id].indices
    pos_item_id = np.random.choice(playlist_seen_items)
    
    neg_item_selected = False
    
    while (not neg_item_selected):
        neg_item_id = np.random.randint(0, n_tracks)
        
        if (neg_item_id not in playlist_seen_items):
            neg_item_selected = True
            
    return playlist_id, pos_item_id, neg_item_id

In [7]:
import scipy.sparse as sps


def similarityMatrixTopK(item_weights, forceSparseOutput = True, k=100, verbose = False, inplace=True):
    """
    The function selects the TopK most similar elements, column-wise

    :param item_weights:
    :param forceSparseOutput:
    :param k:
    :param verbose:
    :param inplace: Default True, WARNING matrix will be modified
    :return:
    """

    assert (item_weights.shape[0] == item_weights.shape[1]), "selectTopK: ItemWeights is not a square matrix"

    start_time = time.time()

    if verbose:
        print("Generating topK matrix")

    nitems = item_weights.shape[1]
    k = min(k, nitems)

    # for each column, keep only the top-k scored items
    sparse_weights = not isinstance(item_weights, np.ndarray)

    if not sparse_weights:

        idx_sorted = np.argsort(item_weights, axis=0)  # sort data inside each column

        if inplace:
            W = item_weights
        else:
            W = item_weights.copy()

        # index of the items that don't belong to the top-k similar items of each column
        not_top_k = idx_sorted[:-k, :]
        # use numpy fancy indexing to zero-out the values in sim without using a for loop
        W[not_top_k, np.arange(nitems)] = 0.0

        if forceSparseOutput:
            W_sparse = sps.csr_matrix(W, shape=(nitems, nitems))

            if verbose:
                print("Sparse TopK matrix generated in {:.2f} seconds".format(time.time() - start_time))

            return W_sparse

        if verbose:
            print("Dense TopK matrix generated in {:.2f} seconds".format(time.time()-start_time))

        return W

    else:
        # iterate over each column and keep only the top-k similar items
        data, rows_indices, cols_indptr = [], [], []

        item_weights = check_matrix(item_weights, format='csc', dtype=np.float32)

        for item_idx in range(nitems):

            cols_indptr.append(len(data))

            start_position = item_weights.indptr[item_idx]
            end_position = item_weights.indptr[item_idx+1]

            column_data = item_weights.data[start_position:end_position]
            column_row_index = item_weights.indices[start_position:end_position]

            non_zero_data = column_data!=0

            idx_sorted = np.argsort(column_data[non_zero_data])  # sort by column
            top_k_idx = idx_sorted[-k:]

            data.extend(column_data[non_zero_data][top_k_idx])
            rows_indices.extend(column_row_index[non_zero_data][top_k_idx])


        cols_indptr.append(len(data))

        # During testing CSR is faster
        W_sparse = sps.csc_matrix((data, rows_indices, cols_indptr), shape=(nitems, nitems), dtype=np.float32)
        W_sparse = W_sparse.tocsr()

        if verbose:
            print("Sparse TopK matrix generated in {:.2f} seconds".format(time.time() - start_time))

        return W_sparse

In [8]:
import time

class SLIM_BPR_Recommender(object):
    """ SLIM_BPR recommender with cosine similarity and no shrinkage"""

    def __init__(self, URM):
        self.URM = URM
        
        self.URM_mask = self.URM.copy()
        self.URM_mask.eliminate_zeros()
        
        self.n_playlists = self.URM_mask.shape[0]
        self.n_tracks = self.URM_mask.shape[1]
        
        self.similarity_matrix = np.zeros((self.n_tracks,self.n_tracks))
        
        self.uneligiblePlaylists = []

        for playlist_id in range(self.n_playlists):

            start_pos = self.URM_mask.indptr[playlist_id]
            end_pos = self.URM_mask.indptr[playlist_id+1]

            if len(self.URM_mask.indices[start_pos:end_pos]) <= 0:
                self.uneligiblePlaylists.append(playlist_id)
        
        # print(self.uneligiblePlaylists)
        # print("Fraction of uneligiblePlaylists playlists: {:.2f}".format(float(len(self.uneligiblePlaylists))/self.n_playlists))


    def sampleTriplet(self):

        # By randomly selecting a user in this way we could end up 
        # with a user with no interactions
        #user_id = np.random.randint(0, n_users)

        playlist_id = np.random.randint(0, n_playlists)
        
        while playlist_id in self.uneligiblePlaylists:
            playlist_id = np.random.randint(0, n_playlists)
    
        playlist_seen_items = URM_train[playlist_id].indices    
        pos_item_id = np.random.choice(playlist_seen_items)
    
        neg_item_selected = False
    
        while (not neg_item_selected):
            neg_item_id = np.random.randint(0, n_tracks)
        
            if (neg_item_id not in playlist_seen_items):
                neg_item_selected = True
            
        return playlist_id, pos_item_id, neg_item_id
        
    def epochIteration(self):

        # Get number of available interactions
        numPositiveIteractions = int(self.URM_mask.nnz*0.01)

        start_time_epoch = time.time()
        start_time_batch = time.time()

        # Uniform user sampling without replacement
        for num_sample in range(numPositiveIteractions):

            # Sample
            playlist_id, positive_item_id, negative_item_id = self.sampleTriplet()

            userSeenItems = self.URM_mask[playlist_id,:].indices

            # Prediction
            x_i = self.similarity_matrix[positive_item_id, userSeenItems].sum()
            x_j = self.similarity_matrix[negative_item_id, userSeenItems].sum()

            # Gradient
            x_ij = x_i - x_j

            gradient = 1 / (1 + np.exp(x_ij))

            # Update
            self.similarity_matrix[positive_item_id, userSeenItems] += self.learning_rate * gradient
            self.similarity_matrix[positive_item_id, positive_item_id] = 0

            self.similarity_matrix[negative_item_id, userSeenItems] -= self.learning_rate * gradient
            self.similarity_matrix[negative_item_id, negative_item_id] = 0


            if(time.time() - start_time_batch >= 30 or num_sample == numPositiveIteractions-1):
                print("Processed {} ( {:.2f}% ) in {:.2f} seconds. Sample per second: {:.0f}".format(
                    num_sample,
                    100.0* float(num_sample)/numPositiveIteractions,
                    time.time() - start_time_batch,
                    float(num_sample) / (time.time() - start_time_epoch)))

                start_time_batch = time.time()

    
    def params(self, learning_rate=0.01):
        self.learning_rate = learning_rate
        
    def optimize(self, epochs = 1):
        for numEpoch in range(epochs):
            self.epochIteration()
            
    def build(self, topK=100):
        self.output_matrix = self.similarity_matrix.T
        self.output_matrix = similarityMatrixTopK(self.output_matrix, k=topK)    
    
    def fit(self, learning_rate=0.01, epochs=10):
        self.learning_rate = learning_rate
        self.epochs = epochs

        print("fit(): starting epochs")
        for numEpoch in range(self.epochs):
            self.epochIteration()
            
        self.similarity_matrix = self.similarity_matrix.T
        
        print("fit(): topK")
        self.similarity_matrix = similarityMatrixTopK(self.similarity_matrix, k=100)
        
        print("fit(): end")
        
        
    def recommend(self, playlist_id, at=None, exclude_seen=True, output=False):
        # compute the scores using the dot product
        playlist = self.URM[playlist_id]
        scores = playlist.dot(self.output_matrix).toarray().ravel()

        if exclude_seen:
            scores = self.filter_seen(playlist_id, scores)

        # rank items
        ranking = scores.argsort()[::-1]
        
        # output for challenge
        if output:
            print("{}, {}".format(playlist_id, " ".join(ranking)))
        
        return ranking[:at]
    
    
    def filter_seen(self, playlist_id, scores):

        start_pos = self.URM.indptr[playlist_id]
        end_pos = self.URM.indptr[playlist_id+1]

        playlist = self.URM.indices[start_pos:end_pos]
        
        scores[playlist] = -np.inf

        return scores

In [10]:
recommender = SLIM_BPR_Recommender(URM_train)
recommender.params(learning_rate=0.1)
recommender.optimize(epochs=30)
recommender.build(topK=100)

Processed 9691 ( 99.99% ) in 2.29 seconds. Sample per second: 4227
Processed 9691 ( 99.99% ) in 1.98 seconds. Sample per second: 4885
Processed 9691 ( 99.99% ) in 2.00 seconds. Sample per second: 4834
Processed 9691 ( 99.99% ) in 1.96 seconds. Sample per second: 4950
Processed 9691 ( 99.99% ) in 2.08 seconds. Sample per second: 4668
Processed 9691 ( 99.99% ) in 1.92 seconds. Sample per second: 5050
Processed 9691 ( 99.99% ) in 2.14 seconds. Sample per second: 4527
Processed 9691 ( 99.99% ) in 2.02 seconds. Sample per second: 4803
Processed 9691 ( 99.99% ) in 1.89 seconds. Sample per second: 5133
Processed 9691 ( 99.99% ) in 1.98 seconds. Sample per second: 4898
Processed 9691 ( 99.99% ) in 1.84 seconds. Sample per second: 5268
Processed 9691 ( 99.99% ) in 1.88 seconds. Sample per second: 5152
Processed 9691 ( 99.99% ) in 1.90 seconds. Sample per second: 5097
Processed 9691 ( 99.99% ) in 1.86 seconds. Sample per second: 5222
Processed 9691 ( 99.99% ) in 1.88 seconds. Sample per second: 

In [ ]:
from Notebooks_utils.evaluation_function import evaluate_algorithm

evaluate_algorithm(URM_test, recommender, at=10)

Evaluated user 0 of 50446
Evaluated user 10000 of 50446
